# RAG to Riches
In this notebook we will apply retrieval-augmented generation (RAG) to the ultimate redemption story (atleast that is available in the public domain) _The Count of Monte Cristo_. He we will leverage two ollama models, `nomic-embed-text` for embedding text and `mistral` for the chat bot. Make sure you have these installed as well as the dependencies below.

In [1]:
import os
import requests

import numpy as np

import ollama

In [2]:
# Lets get the text

url_res = requests.get(url= "https://www.gutenberg.org/cache/epub/1184/pg1184.txt")
fname = 'the_count_of_monte_cristo.txt'
with open(fname, "wb") as f:
    f.write(url_res.content)

In [3]:
# Lets look as a sample of file:

with open(fname, encoding='utf8') as f:
    lines = f.readlines()

np.random.seed(1337)
rand_line_num = np.random.randint(len(lines))

print(lines[rand_line_num:rand_line_num+30])

['possibly be the case, I confess; but if such persons are among my\n', 'acquaintances I prefer not to know it, because then I should be forced\n', 'to hate them.”\n', '\n', '“You are wrong; you should always strive to see clearly around you. You\n', 'seem a worthy young man; I will depart from the strict line of my duty\n', 'to aid you in discovering the author of this accusation. Here is the\n', 'paper; do you know the writing?” As he spoke, Villefort drew the letter\n', 'from his pocket, and presented it to Dantès. Dantès read it. A cloud\n', 'passed over his brow as he said:\n', '\n', '“No, monsieur, I do not know the writing, and yet it is tolerably\n', 'plain. Whoever did it writes well. I am very fortunate,” added he,\n', 'looking gratefully at Villefort, “to be examined by such a man as you;\n', 'for this envious person is a real enemy.” And by the rapid glance that\n', 'the young man’s eyes shot forth, Villefort saw how much energy lay hid\n', 'beneath this mildness.\n', '\n',

In [4]:
# Yikes, that's some odd formating? so let's clean this up:

def lines_to_paragraphs(lines):

    paragraphs = []
    buffer = []
    
    for l in lines:
        
        l = l.strip()
        if l:
            buffer.append(l)
        elif len(buffer):
            paragraphs.append((" ").join(buffer))
            buffer = []
                    
    if len(buffer):
        paragraphs.append((" ").join(buffer))

    return paragraphs
    

paragraphs = lines_to_paragraphs(lines)

In [7]:
# lets find the index of the text previously seen and get the new format

paragraph_lines = []
for i, p in enumerate(paragraphs):
    if lines[rand_line_num].strip() in p:
        paragraph_lines.append(i)

for pl in paragraph_lines:
    print(paragraphs[pl])

print(paragraph_lines)

“You are right; you know men better than I do, and what you say may possibly be the case, I confess; but if such persons are among my acquaintances I prefer not to know it, because then I should be forced to hate them.”
[746]


In [23]:
# Ok, now that the data looks better! Let's embed each paragraph using a pretrained model. 
# Since this may take awhile, lets be able save and load on demand once the embeddings are created.
# For memory and time we will use npy objects.

# One the first time, this may take awhile so grab a coffee.

def load_embeddings(fname):

    inpath = f"embeddings/{fname}.npy"
    if not os.path.exists(f"embeddings/{fname}.npy"):
        return False

    print(f'Loading embeddings from {inpath}')
    with open(f"embeddings/{fname}.npy", "rb") as f:
        embeddings = np.load(f)
    print(f'\t...done!')
    return embeddings


def save_embeddings(embeddings, fname):

    embeddings = np.array(embeddings)
    outpath = f"embeddings/{fname}.npy"

    print(f'saving new embeedings to {outpath}...')
    if not os.path.exists("embeddings"):      
        os.makedirs("embeddings")
  
    with open(outpath, "wb") as f:
        np.save(f, embeddings)
    print(f'\t...done!')


def gen_paragrpah_embeddings(paragraphs, model_name, outname=None):
    
    if (embeddings := load_embeddings(outname)) is not False:
        return embeddings
    
    embeddings = [ollama.embeddings(model=model_name, prompt=p)["embedding"] for p in paragraphs]

    if outname is not None:
        save_embeddings(embeddings, outname)
    
    return embeddings


embeddings = gen_paragrpah_embeddings(paragraphs, "nomic-embed-text", outname='tcomc_embeddings')

saving new embeedings to embeddings/tcomc_embeddings.npy...
	...done!


In [24]:
# now we want to be able to find similar embeddings. For this we will use cosine similarity.
# It is worth while writting this up in a vectorized way:

def ranked_cosine_similarity(ref, embeddings):

    ref = np.array(ref)
    embeddings = np.array(embeddings).T
    cos_similarities = ref.dot(embeddings)/(np.linalg.norm(ref)*np.linalg.norm(embeddings, axis=0))
    return(np.argsort(cos_similarities)[::-1])

cosine_sims_idx = ranked_cosine_similarity(np.array(embeddings[746]), embeddings)

In [25]:
# Let's check that it worked, we should see 746 as the first entry:

cosine_sims_idx[:10]

array([  746, 12739,  6104,  6086, 13831,  2674,  6091, 11459,  4854,
        1933])

In [26]:
# Nice, let's see what else is similar

for p_idx in cosine_sims_idx[:10]:
    print(paragraphs[p_idx])

“You are right; you know men better than I do, and what you say may possibly be the case, I confess; but if such persons are among my acquaintances I prefer not to know it, because then I should be forced to hate them.”
“Hold your tongue! The men are all infamous, and I am happy to be able now to do more than detest them—I despise them.”
“I know it sir,” replied Monte Cristo; “but when I visit a country I begin to study, by all the means which are available, the men from whom I may have anything to hope or to fear, till I know them as well as, perhaps better than, they know themselves. It follows from this, that the king’s attorney, be he who he may, with whom I should have to deal, would assuredly be more embarrassed than I should.”
“Really, sir,” he observed, “I see that in spite of the reputation which you have acquired as a superior man, you look at everything from the material and vulgar view of society, beginning with man, and ending with man—that is to say, in the most restricte

In [27]:
# Cool, getting there! No we write the final RAG function:

RAG_PROMPT = '''
You are a helpful reading assistant who answers questions 
based on snippets of text provided in context. Answer only using the context provided, 
being as concise as possible. If you're unsure, just say that you don't know.
Context:'''

def gen_RAG_prompt(context):

    do_RAG = RAG_PROMPT.replace('\n','')
    
    return f'{do_RAG}\n{context}'
    

def do_RAG(question, embeddings, paragraphs, num_paragraphs=20):

    prompt_embed = ollama.embeddings(model="nomic-embed-text", prompt=question)['embedding']
    similar_paragraph_idxs = ranked_cosine_similarity(prompt_embed, embeddings)
    context = '\n'.join(paragraphs[idx] for idx in similar_paragraph_idxs[:num_paragraphs])
    RAG_prompt = gen_RAG_prompt(context)
    #print(RAG_prompt)

    response = ollama.chat(
        model = 'mistral',
        messages = [{'role': 'system',
                     'content': RAG_prompt},
                    {"role": "user", 
                     "content": question}
                   ]
        )
    return response["message"]["content"]
    

In [28]:
# Ok, now lets test it out:

question = 'How long was Dantes in jail?'
response = do_RAG(question, embeddings, paragraphs)

print(f'Q: {question}')
print(f'A: {response}')

Q: How long was Dantes in jail?
A:  Dantès had been imprisoned for seventeen months in the Château d'If. However, during his time in prison, he had lost track of the passage of time and could not recall exactly how long he had been there when he was brought before the magistrate for a trial. The magistrate, Caderousse, was skeptical of Dantès' story and wanted to know why he had been imprisoned. Dantès recounted his past, from his last voyage as a sailor to his arrest and imprisonment. After finishing his story, Dantès pleaded with Caderousse for a trial, but the magistrate remained unconvinced and left him in the care of another prisoner. The passage suggests that time had passed slowly for Dantès in prison, as he had spent ten months and a half in his cell before being brought before the magistrate.
